In [11]:
import os

In [12]:
%pwd

'c:\\Users\\Akash R\\OneDrive\\Desktop\\Projectts\\Dog-Cat-Classification\\reasearch'

In [13]:
os.chdir("../")

In [14]:
%pwd

'c:\\Users\\Akash R\\OneDrive\\Desktop\\Projectts\\Dog-Cat-Classification'

In [15]:
#entity -> Custom return type for a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [16]:
# Configuration Manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [18]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


In [19]:
# Components
import os
from cnnClassifier import logger
import urllib.request as request
import zipfile
from cnnClassifier.utils.common import get_size

In [20]:
class DataIngestion():
    def __init__(self, config = DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)



In [22]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config  = config.get_data_ingestion_config()
    data_ingestion  = DataIngestion(config = data_ingestion_config )
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-09 16:55:09,375: INFO: common: YAML file: {path} loaded successfully]
[2023-12-09 16:55:09,379: INFO: common: YAML file: {path} loaded successfully]
[2023-12-09 16:55:09,383: INFO: common: created directory at: artifacts]
[2023-12-09 16:55:09,386: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-09 16:55:52,621: INFO: 1413216644: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 18090506
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "e19a17eb599885b5042de89ca380551fbc3e27f2115b609ef58dbc4698bdb4c1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9B2C:2E6D:634CCE:78C00B:65744E98
Accept-Ranges: bytes
Date: Sat, 09 Dec 2023 11:25:15 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10232-MAA
X-Cache: MISS
X-Cache-Hits: 